In [56]:
import dataclasses
import io
import traceback

from matplotlib import pyplot as plt
import numpy as np
import PIL.Image

import sys
sys.path.append('../src/')

import enact
import chat
import llm_task

import asteroids_2goal as asteroids



task_prompt = '''
Write a concise python function with the following signature.
Answer with a single code block.'''
code_gen_prompt = '''

```
def control(position: np.ndarray,
            goal1: np.ndarray,
            goal2: np.ndarray,
            orientation: float) -> Tuple[float, float]:
  """Control an asteroids-style ship to move to both goals.
  
  Args:
    position: A 2D vector representing the position of the ship.
    goal1: A 2D vector representing the position of the first goal.
    goal2: A 2D vector representing the position of the second goal.
    orientation: A float representing the orientation of the ship in radians.
  Returns:
    A pair of floats representing:
      torque: The amount of torque to apply to the ship, between -1 and 1.
      thrust: The amount of thrust to apply to the ship, between 0.0 and 1
  """
  <INSERT IMPLEMENTATION HERE>
```
'''

@enact.typed_invokable(enact.Str, enact.Image)
class PolicyVisualizer(enact.Invokable):
  """Visualizes a policy provided (as a string)."""

  def call(self, code: enact.Str) -> enact.Image:
    """Plots policy trajectories."""
    def_dict = {}
    exec(code, def_dict)
    control = def_dict['control']
    def policy(state: asteroids.State) -> asteroids.Action:
      return asteroids.Action(np.array(list(control(
        state.position[0],
        state.goal1_position[0],
        state.goal2_position[0],
        state.rotation[0])))[np.newaxis])

    _, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
   
    for ax in [ax1, ax2, ax3, ax4]:
      t = asteroids.create_trajectory(policy, (1,), steps=300)
      asteroids.plot_trajectory(t, axis=ax)
    b = io.BytesIO()
    plt.savefig(b, format='png')
    return enact.Image(PIL.Image.open(b))


@enact.typed_invokable(enact.Str, llm_task.ProcessedOutput)
class PolicyChecker(enact.Invokable):

  def call(self, input: enact.Str) -> llm_task.ProcessedOutput:
    if not input.startswith('```python') and not input.startswith('```'):
      return llm_task.ProcessedOutput(
        output=None, correction='Input must start with "```".')
    if not input.endswith('```'):
      return llm_task.ProcessedOutput(
        output=None, correction='Input must end with "```".')
    if '```' in input[3:-3]:
      return llm_task.ProcessedOutput(
        output=None, correction='Input must be a single code block```".')
    if input.startswith('```python'):
      code = input[len('```python`'):-len('```')]
    else:
      code = input[len('```'):-len('```')]
    def_dict = {}
    try:
      exec(code, def_dict)
    except Exception as e:
      return llm_task.ProcessedOutput(
        output=None,
        correction=f'Your code raised an exception while parsing: {e}\n{traceback.format_exc()}')
    control = def_dict.get('control')
    if not control:
      return llm_task.ProcessedOutput(
        output=None,
        correction='Your code did not define a `control` function.')
    try:
      result = control(np.zeros((2,)), np.zeros((2,)), np.zeros((2,)), 0.0)
    except Exception as e:
      return llm_task.ProcessedOutput(
        output=None,
        correction=f'Your code raised an exception while running: {e}\n{traceback.format_exc()}')
    try:
      thrust, torque = result
      thrust = float(thrust)
      torque = float(torque)
    except TypeError:
      return llm_task.ProcessedOutput(
        output=None,
        correction='Your code could not be unpacked into two float values.')
    critique = enact.RequestInput(enact.Str, 'Please critique the policy or leave empty if ok.')(
      PolicyVisualizer()(enact.Str(code)))
  
    if critique != '':
      return llm_task.ProcessedOutput(
        output=None, correction=f'User critique: {critique}')
    
    return llm_task.ProcessedOutput(
      output=code, correction=None)
      

code_gen = llm_task.Task(
  task_prompt=task_prompt,
  chat_agent=chat.GPTAgent(model='gpt-4'))
code_gen.add_example(
  '''```def add(x: int, y: int):\n  <INSERT IMPLEMENTATION HERE>```''',
  '''```python\ndef add(x: int, y: int):\n  return x + y\n```''')

@enact.typed_invokable(enact.NoneResource, enact.Str)
@dataclasses.dataclass
class CreatePolicy(enact.Invokable):
  code_gen: llm_task.Task
  
  def call(self):
    return self.code_gen(enact.Str(code_gen_prompt))


store = enact.Store()
with store:
  code_gen.post_processor = enact.commit(PolicyChecker())
  code_gen.max_retries = 2
  create_policy = CreatePolicy(code_gen)


In [4]:
enact.Registry.get().allow_reregistration=True

In [3]:
with store:
  gui = enact.GUI(
    enact.commit(create_policy),
    input_required_outputs=[enact.Image],
    input_required_inputs=[enact.Str])
  gui.launch(share=True, debug=True)
  #gui.queue().launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://19ae72af12fa24e8fc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://19ae72af12fa24e8fc.gradio.live


In [ ]:
ref_id_str = '{"digest": "a1c474e023fc3a442a79f062c0eb927ff4d47fa37539f9098042b31f2f67c015"}'
with store:
  reference = enact.Ref.from_id(ref_id_str)
  enact.pprint(reference())

Invocation:
  request:
    -> Request#95a158:
      invokable:
        -> CreatePolicy#2156f8:
          code_gen:
            Task:
              task_prompt: '\nWrite a concise python function with the following signature.\nAnswer with a single code block.'
              examples: [ TaskExample(input='```def add(x: int, y: int):\n  <INSERT IMPLEMENTATION HERE>```', output='```python\ndef add(x: int, y: int):\n  return x + y\n```')]
              corrections: []
              post_processor: -> PolicyChecker()#b4a2c7
              max_retries: 2
              chat_agent: GPTAgent(model='gpt-4')
      input: -> NoneResource()#4c96e8
  response:
    -> Response#7def5a:
      invokable:
        -> CreatePolicy#b0ad1d:
          code_gen:
            Task:
              task_prompt: '\nWrite a concise python function with the following signature.\nAnswer with a single code block.'
              examples: [ TaskExample(input='```def add(x: int, y: int):\n  <INSERT IMPLEMENTATION HERE>```',

In [ ]:
print("""python\nimport numpy as np\nfrom typing import Tuple\n\ndef control(position: np.ndarray,\n            goal1: np.ndarray,\n            goal2: np.ndarray,\n            orientation: float) -> Tuple[float, float]:\n  \n  direction = np.mean([goal1, goal2], axis=0) - position\n  angle_to_goal = np.arctan2(direction[1], direction[0])\n  \n  angle_difference = angle_to_goal - orientation\n  angle_difference = (angle_difference + np.pi) % (2 * np.pi) - np.pi  # Normalize to [-pi, pi]\n  \n  torque = np.clip(angle_difference, -1, 1)\n  \n  distance = np.linalg.norm(direction)\n  thrust = np.clip(1 - (distance / np.linalg.norm([goal1, goal2])), 0, 1)\n\n  return torque, thrust\n```', correction='Your code raised an exception while running: control() missing 1 required positional argument: \'orientation\'\nTraceback (most recent call last):\n  File "/tmp/ipykernel_701709/1295529403.py", line 99, in call\n    result = control(np.zeros((2,)), np.zeros((2,)), 0.0)\nTypeError: control() missing 1 required positional argument: \'orientation\'\n""")

python
import numpy as np
from typing import Tuple

def control(position: np.ndarray,
            goal1: np.ndarray,
            goal2: np.ndarray,
            orientation: float) -> Tuple[float, float]:
  
  direction = np.mean([goal1, goal2], axis=0) - position
  angle_to_goal = np.arctan2(direction[1], direction[0])
  
  angle_difference = angle_to_goal - orientation
  angle_difference = (angle_difference + np.pi) % (2 * np.pi) - np.pi  # Normalize to [-pi, pi]
  
  torque = np.clip(angle_difference, -1, 1)
  
  distance = np.linalg.norm(direction)
  thrust = np.clip(1 - (distance / np.linalg.norm([goal1, goal2])), 0, 1)

  return torque, thrust
```', correction='Your code raised an exception while running: control() missing 1 required positional argument: 'orientation'
Traceback (most recent call last):
  File "/tmp/ipykernel_701709/1295529403.py", line 99, in call
    result = control(np.zeros((2,)), np.zeros((2,)), 0.0)
TypeError: control() missing 1 required positional argument:

In [5]:
single__soln = """
import numpy as np
from typing import Tuple

def control(position: np.ndarray,
            goal: np.ndarray,
            orientation: float) -> Tuple[float, float]:
  
  direction = np.arctan2(goal[1] - position[1], goal[0] - position[0])
  torque = direction - orientation

  while torque > np.pi: torque -= 2 * np.pi
  while torque < -np.pi: torque += 2 * np.pi

  torque = np.clip(torque, -1, 1)    

  distance = np.linalg.norm(goal - position)

  if distance > 1:
    thrust = 1
  else:
    thrust = distance

  return torque, thrust
"""

In [17]:
code_prefix = '''import numpy as np\nfrom typing import Tuple\ndef control_one_goal(position: np.ndarray,\n    goal: np.ndarray,\n    orientation: float) -> Tuple[float, float]:\n\n  direction = np.arctan2(goal[1] - position[1], goal[0] - position[0])\n  torque = direction - orientation\n  while torque > np.pi: torque -= 2 * np.pi\n  while torque < -np.pi: torque += 2 * np.pi\n\n  torque = np.clip(torque, -1, 1)\n\n  distance = np.linalg.norm(goal - position)\n  if distance > 1:\n    thrust = 1\n  else:\n    thrust = distance\n  return torque, thrust\n\n'''
print(code_prefix)

import numpy as np
from typing import Tuple
def control_one_goal(position: np.ndarray,
    goal: np.ndarray,
    orientation: float) -> Tuple[float, float]:

  direction = np.arctan2(goal[1] - position[1], goal[0] - position[0])
  torque = direction - orientation
  while torque > np.pi: torque -= 2 * np.pi
  while torque < -np.pi: torque += 2 * np.pi

  torque = np.clip(torque, -1, 1)

  distance = np.linalg.norm(goal - position)
  if distance > 1:
    thrust = 1
  else:
    thrust = distance
  return torque, thrust




In [57]:
task_prompt = '''
Write a concise python function with the following signature.
Answer with a single code block. Call the existing function 
control_one_goal, which has the following signature:

```
def control_one_goal(position: np.ndarray,
                     goal: np.ndarray,
                     orientation: float) -> Tuple[float, float]
  """Control an asteroids-style ship to move to a goal.
  
  Args:
    position: A 2D vector representing the position of the ship.
    goal: A 2D vector representing the position of the goal.
    orientation: A float representing the orientation of the ship in radians.
  Returns:
    A pair of floats representing:
      torque: The amount of torque to apply to the ship, between -1 and 1.
      thrust: The amount of thrust to apply to the ship, between 0.0 and 1
```

Note that this function requires importing numpy as np as well as Tuple from
typing. Do not create a stub for control_one_goal but assume that it is defined
and callable.'''

code_gen_prompt = '''

```
def control(position: np.ndarray,
            goal1: np.ndarray,
            has_reached_goal1: bool,
            goal2: np.ndarray,
            has_reached_goal2: bool,
            orientation: float) -> Tuple[float, float]:
  """Control an asteroids-style ship to move to both goals.
  
  Args:
    position: A 2D vector representing the position of the ship.
    goal1: A 2D vector representing the position of the first goal.
    has_reached_goal1: Boolean representing whether the first goal has been reached.
    goal2: A 2D vector representing the position of the second goal.
    has_reached_goal2: Boolean representing whether the second goal has been reached.
    orientation: A float representing the orientation of the ship in radians.
  Returns:
    A pair of floats representing:
      torque: The amount of torque to apply to the ship, between -1 and 1.
      thrust: The amount of thrust to apply to the ship, between 0.0 and 1
  """
  <INSERT IMPLEMENTATION HERE>
```
'''

@enact.typed_invokable(enact.Str, enact.Image)
class PolicyVisualizer(enact.Invokable):
  """Visualizes a policy provided (as a string)."""

  def call(self, code: enact.Str) -> enact.Image:
    """Plots policy trajectories."""
    def_dict = {}
    exec(code, def_dict)
    control = def_dict['control']
    def policy(state: asteroids.State) -> asteroids.Action:
      return asteroids.Action(np.array(list(control(
        state.position[0],
        state.goal1_position[0],
        state.has_been_at_goal1[0],
        state.goal2_position[0],
        state.has_been_at_goal2[0],
        state.rotation[0])))[np.newaxis])

    _, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
   
    for ax in [ax1, ax2, ax3, ax4]:
      t = asteroids.create_trajectory(policy, (1,), steps=300)
      asteroids.plot_trajectory(t, axis=ax)
    b = io.BytesIO()
    plt.savefig(b, format='png')
    return enact.Image(PIL.Image.open(b))


@enact.typed_invokable(enact.Str, llm_task.ProcessedOutput)
class PolicyChecker(enact.Invokable):

  def call(self, input: enact.Str) -> llm_task.ProcessedOutput:
    if not input.startswith('```python') and not input.startswith('```'):
      return llm_task.ProcessedOutput(
        output=None, correction='Input must start with "```".')
    if not input.endswith('```'):
      return llm_task.ProcessedOutput(
        output=None, correction='Input must end with "```".')
    if '```' in input[3:-3]:
      return llm_task.ProcessedOutput(
        output=None, correction='Input must be a single code block```".')
    if input.startswith('```python'):
      code = input[len('```python`'):-len('```')]
    else:
      code = input[len('```'):-len('```')]
    def_dict = {}
    code = code_prefix + code
    print(code)
    try:
      exec(code, def_dict)
    except Exception as e:
      return llm_task.ProcessedOutput(
        output=None,
        correction=f'Your code raised an exception while parsing: {e}\n{traceback.format_exc()}')
    control = def_dict.get('control')
    if not control:
      return llm_task.ProcessedOutput(
        output=None,
        correction='Your code did not define a `control` function.')
    try:
      result = control(np.zeros((2,)), np.zeros((2,)), 0, np.zeros((2,)), 0, 0.0)
    except Exception as e:
      return llm_task.ProcessedOutput(
        output=None,
        correction=f'Your code raised an exception while running: {e}\n{traceback.format_exc()}')
    try:
      thrust, torque = result
      thrust = float(thrust)
      torque = float(torque)
    except TypeError:
      return llm_task.ProcessedOutput(
        output=None,
        correction='Your code could not be unpacked into two float values.')
    critique = enact.RequestInput(enact.Str, 'Please critique the policy or leave empty if ok.')(
      PolicyVisualizer()(enact.Str(code)))
  
    if critique != '':
      return llm_task.ProcessedOutput(
        output=None, correction=f'User critique: {critique}')
    
    return llm_task.ProcessedOutput(
      output=code, correction=None)
      

code_gen = llm_task.Task(
  task_prompt=task_prompt,
  chat_agent=chat.GPTAgent(model='gpt-4'))
code_gen.add_example(
  '''```def add(x: int, y: int):\n  <INSERT IMPLEMENTATION HERE>```''',
  '''```python\ndef add(x: int, y: int):\n  return x + y\n```''')

@enact.typed_invokable(enact.NoneResource, enact.Str)
@dataclasses.dataclass
class CreatePolicy(enact.Invokable):
  code_gen: llm_task.Task
  
  def call(self):
    return self.code_gen(enact.Str(code_gen_prompt))


store = enact.Store()
with store:
  code_gen.post_processor = enact.commit(PolicyChecker())
  code_gen.max_retries = 2
  create_policy = CreatePolicy(code_gen)


In [58]:
with store:
  ref = enact.commit(create_policy)
  gui = enact.GUI(
    ref,
    input_required_outputs=[enact.Image],
    input_required_inputs=[enact.Str])
  gui.launch(share=True, debug=True)
  #gui.queue().launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://fb70cb73301992c859.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


import numpy as np
from typing import Tuple
def control_one_goal(position: np.ndarray,
    goal: np.ndarray,
    orientation: float) -> Tuple[float, float]:

  direction = np.arctan2(goal[1] - position[1], goal[0] - position[0])
  torque = direction - orientation
  while torque > np.pi: torque -= 2 * np.pi
  while torque < -np.pi: torque += 2 * np.pi

  torque = np.clip(torque, -1, 1)

  distance = np.linalg.norm(goal - position)
  if distance > 1:
    thrust = 1
  else:
    thrust = distance
  return torque, thrust

from typing import Tuple
import numpy as np

def control(position: np.ndarray,
            goal1: np.ndarray,
            has_reached_goal1: bool,
            goal2: np.ndarray,
            has_reached_goal2: bool,
            orientation: float) -> Tuple[float, float]:
  """Control an asteroids-style ship to move to both goals.
  
  Args:
    position: A 2D vector representing the position of the ship.
    goal1: A 2D vector representing the position of the first goal.
  

In [23]:
res = ref.get()
res

NoActiveContext: No context of type Store is active.

In [15]:
new_code_str = '''
import numpy as np
from typing import Tuple
def control_one_goal(position: np.ndarray,
    goal: np.ndarray,
    orientation: float) -> Tuple[float, float]:

  direction = np.arctan2(goal[1] - position[1], goal[0] - position[0])
  torque = direction - orientation
  while torque > np.pi: torque -= 2 * np.pi
  while torque < -np.pi: torque += 2 * np.pi

  torque = np.clip(torque, -1, 1)

  distance = np.linalg.norm(goal - position)
  if distance > 1:
    thrust = 1
  else:
    thrust = distance
  return torque, thrustfrom typing import Tuple
import numpy as np

def control(position: np.ndarray,
            goal1: np.ndarray,
            goal2: np.ndarray,
            orientation: float) -> Tuple[float, float]:
  """Control an asteroids-style ship to move to both goals.
  
  Args:
    position: A 2D vector representing the position of the ship.
    goal1: A 2D vector representing the position of the first goal.
    goal2: A 2D vector representing the position of the second goal.
    orientation: A float representing the orientation of the ship in radians.
  Returns:
    A pair of floats representing:
      torque: The amount of torque to apply to the ship, between -1 and 1.
      thrust: The amount of thrust to apply to the ship, between 0.0 and 1
  """
  torque1, thrust1 = control_one_goal(position, goal1, orientation)
  torque2, thrust2 = control_one_goal(position, goal2, orientation)
  
  # Average the torques and sums the thrusts
  torque = (torque1 + torque2) / 2
  thrust = min(thrust1 + thrust2, 1) #Not to go beyond 1
  
  return torque, thrust
'''

In [16]:
output_dict = {}
exec(new_code_str, output_dict)

SyntaxError: invalid syntax (<string>, line 20)

In [24]:
new_str = '''
import numpy as np
from typing import Tuple
def control_one_goal(position: np.ndarray,
    goal: np.ndarray,
    orientation: float) -> Tuple[float, float]:

  direction = np.arctan2(goal[1] - position[1], goal[0] - position[0])
  torque = direction - orientation
  while torque > np.pi: torque -= 2 * np.pi
  while torque < -np.pi: torque += 2 * np.pi

  torque = np.clip(torque, -1, 1)

  distance = np.linalg.norm(goal - position)
  if distance > 1:
    thrust = 1
  else:
    thrust = distance
  return torque, thrust

from typing import Tuple
import numpy as np

def control_one_goal(position: np.ndarray,
                     goal: np.ndarray,
                     orientation: float) -> Tuple[float, float]:
  """Control an asteroids-style ship to move to a goal.
  
  Args:
    position: A 2D vector representing the position of the ship.
    goal: A 2D vector representing the position of the goal.
    orientation: A float representing the orientation of the ship in radians.
  Returns:
    A pair of floats representing:
      torque: The amount of torque to apply to the ship, between -1 and 1.
      thrust: The amount of thrust to apply to the ship, between 0.0 and 1"""
  
def control(position: np.ndarray,
            goal1: np.ndarray,
            has_reached_goal1: bool,
            goal2: np.ndarray,
            has_reached_goal2: bool,
            orientation: float) -> Tuple[float, float]:
  """Control an asteroids-style ship to move to both goals.
  
  Args:
    position: A 2D vector representing the position of the ship.
    goal1: A 2D vector representing the position of the first goal.
    has_reached_goal1: Boolean representing whether the first goal has been reached.
    goal2: A 2D vector representing the position of the second goal.
    has_reached_goal2: Boolean representing whether the second goal has been reached.
    orientation: A float representing the orientation of the ship in radians.
  Returns:
    A pair of floats representing:
      torque: The amount of torque to apply to the ship, between -1 and 1.
      thrust: The amount of thrust to apply to the ship, between 0.0 and 1"""
      
  if not has_reached_goal1:
    return control_one_goal(position, goal1, orientation)
  elif not has_reached_goal2:
    return control_one_goal(position, goal2, orientation)
  else:
    return (0.0, 0.0)
'''

In [25]:
output_dict = {}
exec(new_str, output_dict)

In [29]:
my_control = output_dict['control']

In [33]:
state = asteroids.State()
output = my_control(
        state.position[0],
        state.goal1_position[0],
        state.has_been_at_goal1[0],
        state.goal2_position[0],
        state.has_been_at_goal2[0],
        state.rotation[0])


IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [40]:
new_str = '''
import numpy as np
from typing import Tuple

def control_one_goal(position: np.ndarray,
                     goal: np.ndarray,
                     orientation: float) -> Tuple[float, float]:
  
  direction = np.arctan2(goal[1] - position[1], goal[0] - position[0])
  torque = direction - orientation

  while torque > np.pi: torque -= 2 * np.pi
  while torque < -np.pi: torque += 2 * np.pi

  torque = np.clip(torque, -1, 1)    

  distance = np.linalg.norm(goal - position)

  if distance > 1:
    thrust = 1
  else:
    thrust = distance

  return torque, thrust

def control(position: np.ndarray,
            goal1: np.ndarray,
            has_reached_goal1: bool,
            goal2: np.ndarray,
            has_reached_goal2: bool,
            orientation: float) -> Tuple[float, float]:
  """Control an asteroids-style ship to move to both goals.
  
  Args:
    position: A 2D vector representing the position of the ship.
    goal1: A 2D vector representing the position of the first goal.
    has_reached_goal1: Boolean representing whether the first goal has been reached.
    goal2: A 2D vector representing the position of the second goal.
    has_reached_goal2: Boolean representing whether the second goal has been reached.
    orientation: A float representing the orientation of the ship in radians.
  Returns:
    A pair of floats representing:
      torque: The amount of torque to apply to the ship, between -1 and 1.
      thrust: The amount of thrust to apply to the ship, between 0.0 and 1
  """
  if not has_reached_goal1:
    return control_one_goal(position, goal1, orientation)
  elif not has_reached_goal2:
    return control_one_goal(position, goal2, orientation)
'''

In [47]:
new_str = '''
import numpy as np
from typing import Tuple
def control_one_goal(position: np.ndarray,
    goal: np.ndarray,
    orientation: float) -> Tuple[float, float]:

  direction = np.arctan2(goal[1] - position[1], goal[0] - position[0])
  torque = direction - orientation
  while torque > np.pi: torque -= 2 * np.pi
  while torque < -np.pi: torque += 2 * np.pi

  torque = np.clip(torque, -1, 1)

  distance = np.linalg.norm(goal - position)
  if distance > 1:
    thrust = 1
  else:
    thrust = distance
  return torque, thrust

def control(position: np.ndarray,
            goal1: np.ndarray,
            has_reached_goal1: bool,
            goal2: np.ndarray,
            has_reached_goal2: bool,
            orientation: float) -> Tuple[float, float]:
  """Control an asteroids-style ship to move to both goals.
  
  Args:
    position: A 2D vector representing the position of the ship.
    goal1: A 2D vector representing the position of the first goal.
    has_reached_goal1: Boolean representing whether the first goal has been reached.
    goal2: A 2D vector representing the position of the second goal.
    has_reached_goal2: Boolean representing whether the second goal has been reached.
    orientation: A float representing the orientation of the ship in radians.
  Returns:
    A pair of floats representing:
      torque: The amount of torque to apply to the ship, between -1 and 1.
      thrust: The amount of thrust to apply to the ship, between 0.0 and 1
  """
  
  if not has_reached_goal1:
    return control_one_goal(position, goal1, orientation)
  elif not has_reached_goal2:
    return control_one_goal(position, goal2, orientation)

'''

output_dict = {}
exec(new_str, output_dict)

In [48]:
output_dict['control']

<function control(position: numpy.ndarray, goal1: numpy.ndarray, has_reached_goal1: bool, goal2: numpy.ndarray, has_reached_goal2: bool, orientation: float) -> Tuple[float, float]>

In [49]:
state = asteroids.State()
output = output_dict['control'](
        state.position[0],
        state.goal1_position[0],
        state.has_been_at_goal1[0],
        state.goal2_position[0],
        state.has_been_at_goal2[0],
        state.rotation[0])


IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [53]:
state.has_been_at_goal1

array(0., dtype=float32)

In [54]:
state.rotation[0]

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed